This notebook describes various setup tasks.

# Repository Setup

Set up makefiles for ``README.rst``.  This file will appear on the bitbucket front page for the project, but the Makefile will allow me to generate the ``README.html`` file locally.

In [1]:
!hg clone ssh://hg@bitbucket.org/mforbes/makefiles .makefiles

requesting all changes
adding changesets
adding manifests
adding file changes
added 45 changesets with 91 changes to 20 files (+1 heads)
updating to branch default
18 files updated, 0 files merged, 0 files removed, 0 files unresolved


In [2]:
!cd .makefiles; hg up README

4 files updated, 0 files merged, 11 files removed, 0 files unresolved


In [3]:
!cp .makefiles/Makefile.example Makefile

In [4]:
!cd .makefiles; subhg setup

Converted to a subhg repository.


In [6]:
%%file README.rst
.. -*- rst -*- -*- restructuredtext -*-

.. This file should be written using the restructure text
.. conventions.  It will be displayed on the bitbucket source page and
.. serves as the documentation of the directory.

GPU GPE
=======

A collection of utilities for solving the Gross-Pitaevskii equations (GPE)
using GPU clusters and other parallel computing resources.

This project is funded by the WSU Seed Grant, and implemented by Ethan Crowell
over the 2014-2015 academic year.

Overwriting README.rst


In [15]:
%%file .hgignore
README.html
.ipynb_checkpoints

syntax: glob
*.pyc
*.pyo

Overwriting .hgignore


In [8]:
!hg st

? .hgignore
? .makefiles/.hgignore
? .makefiles/Makefile
? .makefiles/Makefile.example
? .makefiles/README.rst
? .makefiles/check.mk
? .makefiles/help.mk
? .makefiles/rst2html.mk
? Initial Setup.ipynb
? Makefile
? README.rst


In [9]:
!hg add

adding .hgignore
adding .makefiles/.hgignore
adding .makefiles/Makefile
adding .makefiles/Makefile.example
adding .makefiles/README.rst
adding .makefiles/check.mk
adding .makefiles/help.mk
adding .makefiles/rst2html.mk
adding Initial Setup.ipynb
adding Makefile
adding README.rst


In [10]:
!hg com -m "Initial commit: Makefiles etc."

In [11]:
%%file .hgrc
[paths]
pristine = ssh://hg@bitbucket.org/mforbes/gpu-gpe
mforbes = ssh://hg@bitbucket.org/mforbes/gpu-gpe-fork
ethan = ssh://hg@bitbucket.org/ecrowell4/gpu-gpe-fork

Writing .hgrc


In [12]:
!hg add

adding .hgrc


In [16]:
!hg amend

# Software Requirements

## PyFFTW

While not strictly needed, a key to performance is a good library for computing the FFT, and the FFTW is one of the best.  You will need to compile and install the FFTW library for use, then install PyFFTW.  If you build these yourself, make sure to compile all precisions.  I use commands like this:

```bash
PREFIX=/data/apps/fftw
VER=3.3.4
for opt in " " "--enable-sse2 --enable-single" \
           "--enable-long-double" "--enable-quad-precision"; do
  ./configure --prefix="${PREFIX}/${VER}"\
              --enable-threads\
              --enable-shared\
              $opt
  make -j8 install
done
```

(On Windows, there are PyFFTW installers that include the FFTW libraries:

https://pypi.python.org/pypi/pyFFTW

Install the appropriate version for Python 2.7 and your os.)



In [27]:
import pyfftw
import numpy as np
N = 128
X = np.ones((N,)*3, dtype=complex)
assert np.allclose(
    np.fft.fftn(X), pyfftw.interfaces.numpy_fft.fftn(X))
%timeit np.fft.fftn(X)
pyfftw.interfaces.cache.enable()  # Allow caching of the plans
%timeit pyfftw.interfaces.numpy_fft.fftn(X, threads=8)  # Allow threads - FFTW will measure for best performance

10 loops, best of 3: 140 ms per loop
The slowest run took 66.60 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 17 ms per loop
